In [2]:
import random

from loguru import logger
import sys
logger.remove()
logger.add(sys.stderr, level="INFO")

from itertools import chain, combinations
from dataclasses import dataclass, field
from typing import List, Optional, Tuple

In [9]:
RANKS = {1: 'A', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: 'J', 9: 'Q', 10: 'K'}
SUITS = {'P': '♠', 'F': '♣', 'D': '♢', 'C': '♡'}


@dataclass
class Card:    
    rank: int
    suit: str
        
    def __repr__(self):
        return f"{RANKS[self.rank]}{SUITS[self.suit]}"
    
    def __eq__(self, other):
        return self.rank == other.rank and self.suit == other.suit
    
    def __lt__(self, other):
        _SUITS = list(SUITS.keys())
        return (
            self.rank < other.rank or 
            (self.rank == other.rank and _SUITS.index(self.suit) < _SUITS.index(other.suit))
        )
    
    def __hash__(self):
        return hash(self.__repr__())
    
def make_cards():
    return [Card(rank, suit) for suit in SUITS for rank in RANKS]
        
@dataclass        
class Deck:    
    cards: List[Card] = field(default_factory=make_cards)
    
    def __len__(self):
        return len(self.cards)
    
    def __getitem__(self, position):
        return self.cards[position]
    
    def shuffle(self):
        random.shuffle(self.cards)

class Player:
    
    def __init__(self, name: str):        
        self.name = name
        self.reset()
        
    def reset(self):
        self.hand = []
        self.loot = []
        self.scope = 0        
        
    def __repr__(self):
        return f"{self.name}"
    
    def get_options(self, table):
        return {
            combo: sum(card.rank for card in combo)
            for combo in filter(
                lambda x: 0 < sum(card.rank for card in x) and sum(card.rank for card in x) <= 10, 
                chain.from_iterable(combinations(table, r) for r in range(len(table) + 1))
            )}
    
    def choose_card(self):
        random.shuffle(self.hand)
        return self.hand.pop()        
        
    def play(self, table: list):        
        _options = self.get_options(table)
        chosen_card = self.choose_card()               
        options = {combo: value for combo, value in _options.items() if value == chosen_card.rank}
        if options:
            loot = list(list(options.keys())[0]) # migliorare il criterio di scelta...
            logger.debug(f"{self.name} prende {loot} da {table} giocando {chosen_card}.")
            table = list(set(table) - set(loot))            
            self.loot.extend(loot + [chosen_card])
            if not table:
                logger.debug("Scopa!")
                self.scope += 1
        else:
            table.append(chosen_card)
        return table

class Team:
    
    def __init__(self, name1: str, name2: str):
        self.players = [Player(name1), Player(name2)]
        self.score = 0
        
    def reset(self):
        for player in self.players:
            player.reset()
        
    def __len__(self):
        return len(self.players)
    
    def __getitem__(self, position):
        return self.players[position]
    
    def __repr__(self):
        return f"{self.__class__.__name__}({', '.join(player.__repr__() for player in self.players)})"
    
    def count_scope(self):
        return sum(player.scope for player in self.players)
    
    def compute_score(self):
        logger.info(self)
        scope = self.count_scope()
        if scope:
            logger.info(f"+{scope} scope")
        denari, settebello, carte, primiera, cappotto = 0, 0, 0, 0, 0
        self.loot = self.players[0].loot + self.players[1].loot
        _denari = len(list(filter(lambda x: x.suit == 'D', self.loot)))
        if _denari >= 5:
            if _denari == 5:
                logger.info("Denari pari")
            elif _denari < 10:
                denari = 1
                logger.info("+1 denari")
            if _denari == 10:
                denari = 1
                cappotto = 21
                logger.info("CAPPOTTO!")
        if Card(7, 'D') in self.loot:
            settebello = 1
            logger.info("+1 settebello")
        if len(self.loot) > 20:
            carte = 1
            logger.info("+1 carte")
        elif len(self.loot) == 20:
            logger.info("Carte pari")            
        # implementa primiera        
        _score = scope + denari + settebello + carte + primiera + cappotto
        self.score += _score
        return _score

class Hand:
    
    def __init__(self, teams: Tuple[Team]):
        self.teams = teams
        self.reset()
    
    def reset(self):
        for team in self.teams:
            team.reset()
        self.players = list(player for team in self.teams for player in team)
        first_player = random.choice(tuple(player for player in self.teams[0]))
        second_player = random.choice(tuple(player for player in self.teams[1]))
        self.rotation = (
            first_player, 
            second_player, 
            (set(self.teams[0]) - set([first_player])).pop(),
            (set(self.teams[1]) - set([second_player])).pop()
            )
        self.deck = Deck()
        self.table = []
        self.last_looter: Optional[Player] = None
        
    def distribute_cards(self):        
        cards = self.deck.cards
        for player in self.players:
            player.hand = sorted(random.sample(cards, int(len(self.deck.cards)/len(self.players))))
            cards = tuple(set(cards) - set(player.hand))
            
    def run(self):
        self.reset()
        self.distribute_cards()
        for n in range(len(self.players[0].hand)):
            logger.debug(f"Turno {n + 1}:")
            for player in self.rotation:
                _table = player.play(self.table)                
                if len(_table) < len(self.table):
                    self.last_looter_idx = self.players.index(player)
                self.table = _table
        if self.table:
            logger.info(f"{self.players[self.last_looter_idx]} pulisce il tavolo {self.table}.")
            self.players[self.last_looter_idx].loot.extend(self.table)            
            self.table = []
            
        for player in self.players:
            logger.info(f"{player.name}: {', '.join(card.__repr__() for card in player.loot)}")
        
        return {team: team.compute_score() for team in match.teams}
            
class Match:
    
    def __init__(self, team1: tuple, team2: tuple):
        self.teams = {Team(*team1): 0, Team(*team2): 0}
        self.goal = 21
        
    def run(self):
        while all(score < self.goal for score in self.teams.values()):
            hand = Hand(tuple(self.teams.keys()))
            scores = hand.run()
            for team in self.teams:
                self.teams[team] += scores[team]
        logger.info(self.teams)

In [10]:
match = Match(('A', 'B'), ('C', 'D'))

In [11]:
match.run()

2021-03-28 00:50:20.694 | INFO     | __main__:run:176 - D pulisce il tavolo [Q♠, 2♠, 3♣].
2021-03-28 00:50:20.694 | INFO     | __main__:run:181 - A: 6♣, 4♠, K♣, 6♠, A♢, 7♡
2021-03-28 00:50:20.694 | INFO     | __main__:run:181 - B: 4♡, 4♢, A♡, J♡, Q♣
2021-03-28 00:50:20.694 | INFO     | __main__:run:181 - C: 7♢, A♣, J♣, 6♢, 6♡, K♢, K♡, 3♡, 3♠
2021-03-28 00:50:20.694 | INFO     | __main__:run:181 - D: 2♡, 3♢, 5♡, Q♡, Q♢, 5♢, A♠, 4♣, K♠, J♢, J♠, 2♢, 2♣, 7♠, 7♣, 5♠, 5♣, Q♠, 2♠, 3♣
2021-03-28 00:50:20.694 | INFO     | __main__:compute_score:107 - Team(A, B)
2021-03-28 00:50:20.708 | INFO     | __main__:compute_score:107 - Team(C, D)
2021-03-28 00:50:20.709 | INFO     | __main__:compute_score:110 - +3 scope
2021-03-28 00:50:20.710 | INFO     | __main__:compute_score:119 - +1 denari
2021-03-28 00:50:20.712 | INFO     | __main__:compute_score:126 - +1 settebello
2021-03-28 00:50:20.713 | INFO     | __main__:compute_score:129 - +1 carte
2021-03-28 00:50:20.722 | INFO     | __main__:run:176 - D 